In [1]:
import ee
ee.Initialize()
from iteru import *
from IPython.display import display
from ipywidgets import *
import requests
import os

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [43]:
start_year = 2015
start_month = 9
start_day = 1
end_year = 2022
end_month = 1
end_day =  1
time_delta = 60

In [44]:
start_date = datetime.date(start_year,start_month,start_day)
end_date = datetime.date(end_year,end_month,end_day)

In [45]:
dates_list = get_dates_list(start_date ,end_date, time_delta)

In [46]:
aoi = Map.aoi

In [47]:
col1 = ee.ImageCollection("COPERNICUS/S2")
col2 = ee.ImageCollection("COPERNICUS/S2_SR")

In [48]:
def rename_bands(img):
    return img.select(["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12", "QA60"],
                      ["Blue","Green","Red","Red Edge 1",
                       "Red Edge 2","Red Edge 3","NIR","Red Edge 4","SWIR1","SWIR2","QA60",])

In [49]:
 def fmask(img):
        cloudOpaqueBitMask = 1 << 10
        cloudCirrusBitMask = 1 << 11
        qa = img.select("QA60")
        mask = (
            qa.bitwiseAnd(cloudOpaqueBitMask)
            .eq(0)
            .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
        )
        return img.updateMask(mask)

In [50]:
def prepare_images(img):
    img = rename_bands(img)
    if fmask:
        img = fmask(img)
    return ee.Image(img)

In [51]:
col1 = col1.filterBounds(aoi).map(prepare_images)
col2 = col2.filterBounds(aoi).map(prepare_images)
col = col1.merge(col2)

In [52]:
def get_images(day_date):
    
    start_day = ee.Date(day_date)
    end_day = start_day.advance(time_delta,'day')
    
    return col.filterDate(start_day,end_day).reduce(ee.Reducer.median())

In [53]:
images = ee.List(dates_list).map(get_images)

In [54]:
collection = ee.ImageCollection.fromImages(images)

In [55]:
videoArgs = {
  'dimensions': 800,
  'region': aoi,
  'framesPerSecond': 3,
  'crs': 'EPSG:3857',
  'min': 0,
  'max': 3500,
  'bands':['Red_median','Green_median','Blue_median']
}

In [56]:
url = collection.getVideoThumbURL(videoArgs)

In [57]:
r = requests.get(url, stream=True)

In [58]:
def random_string(string_length=4):
    import random
    import string
    letters = string.ascii_uppercase
    return "".join(random.choice(letters) for i in range(string_length))

In [59]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
filename = "TimeSeries_" + random_string() + ".gif"
out_gif = os.path.join(out_dir, filename)

In [60]:
with open (out_gif,'wb') as file:
    for chunk in r.iter_content(chunk_size=1024):
        file.write(chunk)

In [61]:
from PIL import Image, ImageDraw, ImageFont, ImageSequence
import io

In [62]:
ximage = Image.open(out_gif)

In [63]:
count = ximage.n_frames

In [64]:
W, H = ximage.size

In [65]:
xy = (int(0.05 * W), int(0.05 * H))

In [66]:
text = dates_list

In [67]:
frames = []

In [68]:
font = ImageFont.truetype(r'C:\Users\muham\Downloads\News 705 Italic BT\News 705 Italic BT.ttf', 30)

In [69]:
for index, frame in enumerate(ImageSequence.Iterator(ximage)):
    frame = frame.convert("RGB")
    draw = ImageDraw.Draw(frame)
    draw.text(xy, text[index],fill ="blue",font = font)
    b = io.BytesIO()
    frame.save(b, format="GIF")
    frame = Image.open(b)
    frames.append(frame)
    

In [70]:
frames[0].save(
            out_gif,
            save_all=True,
            append_images=frames[1:],
            duration=int(1000/videoArgs['framesPerSecond']),
            loop=0,
            optimize=True,
        )

In [71]:
out = Output()
out.clear_output(wait=True)
display(out)
with out:
    with open(out_gif,'rb')  as file :
        image = file.read()
    #display(Image(value=image))

Output()